In [2]:
import sys
import os
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from logger_config import logger

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver


from dotenv import load_dotenv

load_dotenv()

True

In [3]:
logger.info("hi")

2025-05-15 22:04:56 | 480633797.py | <module> | Line: 1 | INFO | hi


In [4]:
memory = MemorySaver()

In [5]:
class State(TypedDict):
    value: str

In [15]:
def node_a(state: State): 
    logger.info("Node A")
    return Command(
        goto="node_b", 
        update={
            "value": state["value"] + "a"
        }
    )

def node_b(state: State): 
    logger.info("Node B")

    human_response = interrupt("Do you want to go to C or D? Type C/D")

    logger.info(f"Human Review Values: {human_response}")
    
    if(human_response == "C"): 
        return Command(
            goto="node_c", 
            update={
                "value": state["value"] + "b"
            }
        ) 
    elif(human_response == "D"): 
        return Command(
            goto="node_d", 
            update={
                "value": state["value"] + "b"
            }
        )


def node_c(state: State): 
    logger.info("Node C")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def node_d(state: State): 
    logger.info("Node D")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)


In [19]:
graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a") 

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

initialState = {
    "value": ""
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result

2025-05-15 22:09:23 | 2973161331.py | node_a | Line: 2 | INFO | Node A
2025-05-15 22:09:23 | 2973161331.py | node_b | Line: 11 | INFO | Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:aefcf7eb-3367-5db1-34e9-930dd9f03d5c']),)}]

In [20]:
logger.info(app.get_state(config).next)

2025-05-15 22:09:24 | 3385816404.py | <module> | Line: 1 | INFO | ('node_b',)


In [21]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

2025-05-15 22:09:24 | 2973161331.py | node_b | Line: 11 | INFO | Node B
2025-05-15 22:09:24 | 2973161331.py | node_b | Line: 15 | INFO | Human Review Values: C
2025-05-15 22:09:24 | 2973161331.py | node_c | Line: 34 | INFO | Node C


[{'node_b': {'value': 'ab'}}, {'node_c': {'value': 'abc'}}]